<a href="https://colab.research.google.com/github/Cobord/Quantum-Computer-Things/blob/master/Open%20Systems/depolarization_channel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [20]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

num_qubits=10
matrix_dim=2**num_qubits
lambda_depolarize=.1

with tf.device('/cpu:0'):
  state_helperC = tf.cast(tf.random_normal((matrix_dim, matrix_dim)),dtype=tf.complex64)
  state_helperC=state_helperC+tf.scalar_mul(tf.constant(1.0j,dtype=tf.complex64),tf.cast(tf.random_normal((matrix_dim, matrix_dim)),dtype=tf.complex64))
  stateC=tf.matmul(tf.linalg.adjoint(state_helperC),state_helperC)
  stateC=stateC/tf.trace(stateC)
  depolarize_change_helperC=tf.constant((1-lambda_depolarize)/matrix_dim,dtype=tf.complex64)
  depolarize_changeC=tf.scalar_mul(depolarize_change_helperC,tf.linalg.LinearOperatorIdentity(num_rows=matrix_dim,dtype=tf.complex64).to_dense())
  depolarizeC=tf.scalar_mul(lambda_depolarize,stateC)+depolarize_changeC

with tf.device('/gpu:0'):
  state_helperG = tf.cast(tf.random_normal((matrix_dim, matrix_dim)),dtype=tf.complex64)
  state_helperG=state_helperG+tf.scalar_mul(tf.constant(1.0j,dtype=tf.complex64),tf.cast(tf.random_normal((matrix_dim, matrix_dim)),dtype=tf.complex64))
  stateG=tf.matmul(tf.linalg.adjoint(state_helperG),state_helperG)
  stateG=stateG/tf.trace(stateG)
  depolarize_change_helperG=tf.constant((1-lambda_depolarize)/matrix_dim,dtype=tf.complex64)
  depolarize_changeG=tf.scalar_mul(depolarize_change_helperG,tf.linalg.LinearOperatorIdentity(num_rows=matrix_dim,dtype=tf.complex64).to_dense())
  depolarizeG=tf.scalar_mul(lambda_depolarize,stateG)+depolarize_changeG
  
sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(depolarizeC)
  
def gpu():
  sess.run(depolarizeG)
  
# Runs the op several times.
my_number=10
print('Time (s) to compute \rho and \Delta_\lambda (\rho) for %d qubits. Average of %d runs.'%(num_qubits,my_number))
cpu_time = timeit.timeit('cpu()', number=my_number, setup="from __main__ import cpu")
print('CPU (s): %.5f'%(cpu_time/my_number))
gpu_time = timeit.timeit('gpu()', number=my_number, setup="from __main__ import gpu")
print('GPU (s): %.5f'%(gpu_time/my_number))
print('GPU vs CPU: %.5f'%(cpu_time/gpu_time))

sess.close()

ho) for 10 qubits. Average of 10 runs.
CPU (s): 0.31319
GPU (s): 0.01439
GPU vs CPU: 21.75739
